In [37]:
import numpy as np
import math
import itertools
import time
from multiprocessing import Pool
import scipy.sparse
from IPython.display import clear_output
from multiprocessing.dummy import Pool as ThreadPool
import csv
import ast

In [2]:
#Given an omino (nxn binary matrix), checks if it is valid
# i.e. all ones are rook-adjacent and contiguous

def valid_omino(omino):
    k = omino.shape[0]
    check = np.zeros([k,k]) - omino
    
    
    _f  = False
    for i in range(k):
        for j in range(k):
            if check[i,j] == -1:
                if not _f:
                    _f  = True
                    check[i,j] = 1
    if not _f:
        print("ABORT")
        return
    
    for iter in range(15):
        for i in range(k):
            for j in range(k):
                if check[i,j] == 1:
                    if i != 0:
                        check[i-1,j] = check[i-1,j]**2


                    if i != k-1:
                        check[i+1,j] = check[i+1,j]**2

                    if j != 0:
                        check[i,j-1] = check[i,j-1]**2

                    if j != k-1:
                        check[i,j+1] = check[i,j+1]**2
                    
    return np.sum(check) == np.sum(omino)
                        
    



In [3]:
# given a list of (r,c) index pairs, generates an omino
# if the omino isn't valid, returns the 0x0 0 matrix
def make_omino(inds, dim):
    dim
    p = np.zeros([dim,dim])
    for loc in inds:
        p[loc[0],loc[1]] = 1
    
    if valid_omino(p):
        return p
    else:
        return np.zeros([0,0])

In [4]:
# generates all of the <cells>-ominos in the <grid>^2 grid
# WARNING - not tested for cells != grid
def make_omino_set(cells, grid):
    
    
    if grid % cells != 0:
        print("WARNING - <cells> must divide <grid>")
        #return []
    
    
    pair_idx = []
    for i in range(grid):
        for j in range(grid):
            pair_idx.append((i,j))
    
    
    ominos = []
    
    for t in (l for l in itertools.combinations(pair_idx, cells)):
        p = make_omino(t,grid)
        if p.shape[0] != 0:
            ominos.append(p)
    
    return ominos
        

In [5]:
# checks if an omino is queen-contiguity valid
def valid_omino_qc(omino):
    k = omino.shape[0]
    check = np.zeros([k,k]) - omino
    
    
    _f  = False
    for i in range(k):
        for j in range(k):
            if check[i,j] == -1:
                if not _f:
                    _f  = True
                    check[i,j] = 1
    if not _f:
        print("ABORT")
        return
    
    for iter in range(15):
        for i in range(k):
            for j in range(k):
                if check[i,j] == 1:
                    if i != 0:
                        check[i-1,j] = check[i-1,j]**2


                    if i != k-1:
                        check[i+1,j] = check[i+1,j]**2

                    if j != 0:
                        check[i,j-1] = check[i,j-1]**2

                    if j != k-1:
                        check[i,j+1] = check[i,j+1]**2
                        
                    if i!= 0 and j!=0:
                        check[i-1,j-1] = check[i-1,j-1]**2
                        
                    if i!=0 and j!= k-1:
                        check[i-1,j+1] = check[i-1,j+1]**2
                        
                    if i != k-1 and j != 0:
                        check[i+1,j-1] = check[i+1,j-1]**2
                   
                    if j != k-1 and i != 0:
                        check[i-1,j+1] = check[i-1,j+1]**2
                        
                    if i!= k-1 and j != k-1:
                        check[i+1,j+1] = check[i+1,j+1]**2
                    
                    
    return np.sum(check) == np.sum(omino)

In [6]:
# given a list of (r,c) index pairs, generates a queen
# contiguous omino
# if the omino isn't valid, returns the 0x0 0 matrix
def make_omino_qc(inds, dim):
    dim
    p = np.zeros([dim,dim])
    for loc in inds:
        p[loc[0],loc[1]] = 1
    
    if valid_omino_qc(p):
        return p
    else:
        return np.zeros([0,0])

In [7]:
#makes queen continguous 'ominos'
def make_omino_set_qc(cells, grid):
    
    
    if grid % cells != 0:
        print("WARNING - <cells> must divide <grid>")
        #return []
    
    
    pair_idx = []
    for i in range(grid):
        for j in range(grid):
            pair_idx.append((i,j))
    
    
    ominos = []
    
    for t in (l for l in itertools.combinations(pair_idx, cells)):
        p = make_omino_qc(t,grid)
        if p.shape[0] != 0:
            ominos.append(p)
    
    return ominos
        

In [8]:
# make the 4x4 ominos
n = make_omino_set_qc(4,4)
print(len(n))
print([np.sum(x) for x in n if np.sum(x) != 4])
inds = len(n)

419
[]


In [9]:
inds = range(len(n))
tic = time.time()
pairs_inds = [x for x in itertools.combinations(inds,2) if 2 not in n[x[0]]+n[x[1]]]



def make_trips(inx):
    val_trips = []
    counter = 0
    global pairs_inds
    for d in pairs_inds:
        c=inx
        counter +=1
        #if counter%10000 == 0: print(counter/10000)
        q = set(d+(c,))
        if len(q) == 3 and c>max(d):
            check  = sum([n[i] for i in q])
            if 2 not in check: val_trips.append(tuple(q))
    #print("returning on ",inx)
    return val_trips



def make_quads(inx):
    val_quads = []
    counter = 0
    global trips_inds
    for t in trips_inds:
        c=inx
        counter +=1
        #if counter%10000 == 0: print(counter/10000)
        p = set(t+(c,))
        if len(p) == 4 and c>max(t):
            check  = sum([n[i] for i in p])
            if 2 not in check: val_quads.append(tuple(p))
    #if inx%10==0: print("returning on ",inx)
    return val_quads

pool = Pool(8)
results = pool.map(make_trips,inds)
trips_inds  = [item for sublist in results for item in sublist]
trips_inds = list(set([tuple(sorted(j)) for j in trips_inds]))
pool = Pool(8)
results = pool.map(make_quads,inds)
quads_inds = [item for sublist in results for item in sublist]
quads_inds = list(set([tuple(sorted(j)) for j in quads_inds]))

print(time.time()-tic)


36.708109617233276


In [41]:
print(len(quads_inds))

partitions = [[n[i].astype(int).tolist() for i in p] for p in quads_inds]
print(len(partitions))
#print(partitions[3])
with open('parts_4-4_qc.csv','w') as f:
    wr = csv.writer(f)
    wr.writerows(partitions)
l = []    
with open('parts_4-4_qc.csv','r') as f:
    reader = csv.reader(f)
    l = list(reader)
        
partitions = [[ast.literal_eval(t) for t in a] for a  in l]


2620
2620
<class 'list'>


In [8]:
n = make_omino_set(3,4)
n += make_omino_set(4,4)
n += make_omino_set(5,4)
print(len(n))
print(n[0])
print(n[-1])

inds = len(n)






inds = range(len(n))
tic = time.time()
pairs_inds = [x for x in itertools.combinations(inds,2) if 2 not in n[x[0]]+n[x[1]]]



def make_trips(inx):
    val_trips = []
    counter = 0
    global pairs_inds
    for d in pairs_inds:
        c=inx
        counter +=1
        #if counter%10000 == 0: print(counter/10000)
        q = set(d+(c,))
        if len(q) == 3 and c>max(d):
            check  = sum([n[i] for i in q])
            if 2 not in check: val_trips.append(tuple(q))
    #print("returning on ",inx)
    return val_trips



def make_quads(inx):
    val_quads = []
    counter = 0
    global trips_inds
    for t in trips_inds:
        c=inx
        counter +=1
        #if counter%10000 == 0: print(counter/10000)
        p = set(t+(c,))
        if len(p) == 4 and c>max(t):
            check  = sum([n[i] for i in p])
            if 2 not in check and 0 not in check: val_quads.append(tuple(p))
    #if inx%10==0: print("returning on ",inx)
    return val_quads

pool = Pool(8)
results = pool.map(make_trips,inds)
trips_inds  = [item for sublist in results for item in sublist]
trips_inds = list(set([tuple(sorted(j)) for j in trips_inds]))
pool = Pool(8)
results = pool.map(make_quads,inds)
quads_inds = [item for sublist in results for item in sublist]
quads_inds = list(set([tuple(sorted(j)) for j in quads_inds]))

print(time.time()-tic)


WARNING - <cells> must divide <grid>
WARNING - <cells> must divide <grid>
409
[[1. 1. 1. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 1.]
 [1. 1. 1. 1.]]
164.3835563659668


Traceback (most recent call last):
Process ForkPoolWorker-8:
Process ForkPoolWorker-16:
  File "/home/zachary/miniconda3/envs/vrdi/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process ForkPoolWorker-14:
Process ForkPoolWorker-11:
Traceback (most recent call last):
Process ForkPoolWorker-13:
Process ForkPoolWorker-12:
Process ForkPoolWorker-6:
Process ForkPoolWorker-5:
Process ForkPoolWorker-2:
Process ForkPoolWorker-1:
Process ForkPoolWorker-7:
Process ForkPoolWorker-3:
Process ForkPoolWorker-15:
Process ForkPoolWorker-10:
Process ForkPoolWorker-9:
  File "/home/zachary/miniconda3/envs/vrdi/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/zachary/miniconda3/envs/vrdi/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Process ForkPoolWorker-4:
Traceback (most recent call last):
Trac

  File "/home/zachary/miniconda3/envs/vrdi/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/zachary/miniconda3/envs/vrdi/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/zachary/miniconda3/envs/vrdi/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/home/zachary/miniconda3/envs/vrdi/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/zachary/miniconda3/envs/vrdi/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/zachary/miniconda3/envs/vrdi/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/zachary/miniconda3/envs/vrdi/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/zachary/miniconda3/envs/vrdi/lib/python3.6/multiprocessing/connection.py", line 216

In [9]:
print(len(quads_inds))

partitions = [[n[i].astype(int).tolist() for i in p] for p in quads_inds]
print(len(partitions))
print(partitions[3])
with open('parts_4-4_w35_rc.csv','w') as f:
    wr = csv.writer(f)
    wr.writerows(partitions)

1953
1953
[[[0, 0, 0, 0], [1, 0, 0, 0], [1, 1, 0, 0], [0, 0, 0, 0]], [[1, 1, 1, 0], [0, 1, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1]], [[0, 0, 0, 0], [0, 0, 1, 0], [0, 0, 1, 0], [1, 1, 1, 0]]]


In [ ]:

tic = time.time()

n = make_omino_set_qc(3,4)
n += make_omino_set_qc(4,4)
n += make_omino_set_qc(5,4)
print(len(n))
print(n[0])
print(n[-1])

inds = len(n)




print(time.time()-tic)
tic = time.time()

inds = range(len(n))
tic = time.time()
pairs_inds = [x for x in itertools.combinations(inds,2) if 2 not in n[x[0]]+n[x[1]]]


ones = np.ones([4,4])
def make_trips(inx):
    val_trips = []
    counter = 0
    global pairs_inds
    for d in pairs_inds:
        c=inx
        counter +=1
        #if counter%10000 == 0: print(counter/10000)
        q = set(d+(c,))
        if len(q) == 3 and c>max(d):
            check  = sum([n[i] for i in q])
            if 2 not in check and valid_omino_qc(ones-check): val_trips.append(tuple(q))
    #print("returning on ",inx)
    return val_trips


print(time.time()-tic)
tic = time.time()
def make_quads(inx):
    val_quads = []
    counter = 0
    global trips_inds
    for t in trips_inds:
        c=inx
        counter +=1
        #if counter%10000 == 0: print(counter/10000)
        p = set(t+(c,))
        if len(p) == 4 and c>max(t):
            check  = sum([n[i] for i in p])
            if 2 not in check and 0 not in check: val_quads.append(tuple(p))
    #if inx%10==0: print("returning on ",inx)
    return val_quads

pool = Pool(8)
results = pool.map(make_trips,inds)
trips_inds  = [item for sublist in results for item in sublist]
trips_inds = list(set([tuple(sorted(j)) for j in trips_inds]))
pool = Pool(8)
results = pool.map(make_quads,inds)
quads_inds = [item for sublist in results for item in sublist]
quads_inds = list(set([tuple(sorted(j)) for j in quads_inds]))

print(time.time()-tic)




print(len(quads_inds))

partitions = [[n[i].astype(int).tolist() for i in p] for p in quads_inds]
print(len(partitions))
print(partitions[3])
with open('parts_4-4_w35_qc.csv','w') as f:
    wr = csv.writer(f)
    wr.writerows(partitions)

WARNING - <cells> must divide <grid>
WARNING - <cells> must divide <grid>
1707
[[1. 1. 1. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 1.]
 [1. 1. 1. 1.]]
1.6077415943145752
5.826982498168945


In [ ]:
# do the 5x5
n = make_omino_set(5,5)
print(len(n))
print([np.sum(x) for x in n if np.sum(x) != 5])

In [ ]:



inds = range(len(n))
tic = time.time()
pairs_inds = [x for x in itertools.combinations(inds,2) if 2 not in n[x[0]]+n[x[1]]]


print(time.time()-tic)

In [ ]:
tic = time.time()
def make_trips(inx):
    val_trips = []
    counter = 0
    global pairs_inds
    for d in pairs_inds:
        c=inx
        counter +=1
        #if counter%10000 == 0: print(counter/10000)
        q = set(d+(c,))
        if len(q) == 3 and c>max(d):
            check  = sum([n[i] for i in q])
            if 2 not in check: val_trips.append(tuple(q))
    #print("returning on ",inx)
    return val_trips
pool = ThreadPool(8)
results = pool.map(make_trips,inds)
trips_inds  = [item for sublist in results for item in sublist]
trips_inds = list(set([tuple(sorted(j)) for j in trips_inds]))
print(time.time()-tic, len(trips_inds))

In [ ]:
tic = time.time()
def make_quads(inx):
    val_quads = []
    counter = 0
    global trips_inds
    for t in trips_inds:
        c=inx
        counter +=1
        #if counter%10000 == 0: print(counter/10000)
        p = set(t+(c,))
        if len(p) == 4 and c>max(t):
            check  = sum([n[i] for i in p])
            if 2 not in check: val_quads.append(tuple(p))
    if inx%10==0: print("returning on ",inx)
    return val_quads
pool = ThreadPool(8)
results = pool.map(make_quads,inds)
quads_inds = [item for sublist in results for item in sublist]
quads_inds = list(set([tuple(sorted(j)) for j in quads_inds]))
print(time.time()-tic, len(quads_inds))

In [ ]:
val_quads_check = []
val_quads_e = list(set([tuple(sorted(q)) for q in quads_inds]))
print(len(val_quads_e))
counter = 0
ones = np.ones([5,5])
tic = time.time()
for q in val_quads_e:
    counter += 1
    if counter%10000 == 0: print(counter/10000)
    check = sum([n[i] for i in q])
    if valid_omino(ones-check): val_quads_check.append(q)
print(time.time()-tic, len(val_quads_check))

In [ ]:
tic = time.time()
valid_partitions = []
counter = 0
for q in val_quads_check:
    for inx in inds:
        counter +=1
        if counter%1000000 == 0: print(counter/1000000)
        if inx>max(q):
            p = q +(inx,)
            check  = sum([n[i] for i in p])
            if 2 not in check: valid_partitions.append(tuple(p))
print(time.time()-tic, len(valid_partitions))

In [ ]:
np.save("parts_5-5",valid_partitions)
